# Seminar - JSON, XML, Requests, Web-Scraping 
by Vítek Macháček

## Task 1: JSON API and Quandl

Set Quandl access credentials

In [2]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time

QUANDL_API_KEY = 'jsVA9hLwyzY_Pg21uzR4'


Request Quandl for time series:

hint: The most simple Quandl Format for API-call is:

https://www.quandl.com/api/v3/datasets/{QUANDL_DATABASE}/{QUANDL_DATASET}/data.{DATA_FORMAT}?api_key={YOUR-QUANDL-API-KEY}

QUANDL_API_KEY='jsVA9hLwyzY_Pg21uzR4'Let's ask for `FB` dataset from `WIKI` database. The results should be in JSON format

In [6]:
dataset = 'FB'
database = 'WIKI'
form = 'json'
url = f'https://www.quandl.com/api/v3/datasets/{database}/{dataset}/data.{form}?api_key={QUANDL_API_KEY}'#.format(database,dataset,'json',QUANDL_API_KEY)

r = requests.get(url)

In [9]:
type(r.text)

str

convert response to python natives

In [14]:
d = r.json()

What keys does a dictionary contain?

In [16]:
d.keys()

dict_keys(['dataset_data'])

Let's go deeper

In [18]:
d['dataset_data'].keys()

dict_keys(['limit', 'transform', 'column_index', 'column_names', 'start_date', 'end_date', 'frequency', 'data', 'collapse', 'order'])

We want to see the data

In [23]:
d['dataset_data']['column_names']

['Date',
 'Open',
 'High',
 'Low',
 'Close',
 'Volume',
 'Ex-Dividend',
 'Split Ratio',
 'Adj. Open',
 'Adj. High',
 'Adj. Low',
 'Adj. Close',
 'Adj. Volume']

first data point?

In [21]:
d['dataset_data']['data'][0]

['2018-03-27',
 156.31,
 162.85,
 150.75,
 152.19,
 76787884.0,
 0.0,
 1.0,
 156.31,
 162.85,
 150.75,
 152.19,
 76787884.0]

Let's do a DataFrame

In [26]:
pd.DataFrame(d['dataset_data']['data'],columns=d['dataset_data']['column_names'])

,Date,Open,High,Low,Close,Volume,Ex-Dividend,Split Ratio,Adj. Open,Adj. High,Adj. Low,Adj. Close,Adj. Volume
0,2018-03-27,156.31,162.85,150.75,152.1900,76787884.0,0.0,1.0,156.31,162.85,150.75,152.1900,76787884.0
1,2018-03-26,160.82,161.10,149.02,160.0600,125438294.0,0.0,1.0,160.82,161.10,149.02,160.0600,125438294.0
2,2018-03-23,165.44,167.10,159.02,159.3900,52306891.0,0.0,1.0,165.44,167.10,159.02,159.3900,52306891.0
3,2018-03-22,166.13,170.27,163.72,164.8900,73389988.0,0.0,1.0,166.13,170.27,163.72,164.8900,73389988.0
4,2018-03-21,164.80,173.40,163.30,169.3900,105350867.0,0.0,1.0,164.80,173.40,163.30,169.3900,105350867.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1467,2012-05-24,32.95,33.21,31.77,33.0300,50237200.0,0.0,1.0,32.95,33.21,31.77,33.0300,50237200.0
1468,2012-05-23,31.37,32.50,31.36,32.0000,73600000.0,0.0,1.0,31.37,32.50,31.36,32.0000,73600000.0
1469,2012-05-22,32.61,33.59,30.94,31.0000,101786600.0,0.0,1.0,32.61,33.59,30.94,31.0000,101786600.0
1470,2012-05-21,36.53,36.66,33.00,34.0300,168192700.0,0.0,1.0,36.53,36.66,33.00,34.0300,168192700.0


Perhaps column names can be useful

## Task 1b: XML (not covered on seminar)
Request XML format of the same dataset

Let just save a XML into a file `xml_sample.xml`

get BeatifulSoup object

OK gimme a dataframe

## Task 2: Quandl website

Let's get a list of available datasets on Quandl:

The list of datasets on Quandl website is available at: https://www.quandl.com/search?filters=%5B%22Free%22%5D

Is this website scrapable? 

## Task 3: Scraping IES news

use following code snippets to construct your own IES News web scraper

In [33]:
def get_soup(link):
    '''
    Function accepts a link and returns a BeautifulSoup object parsed from text of a succesful GET request on a link. If requests returns other status code than 200, returns None and prints a message

    Make sure that the request object is parsed as UTF-8 string.
    '''
    r = requests.get(link)
    r.encoding = 'utf-8'
    if r.status_code == 200:
        return BeautifulSoup(r.text)
    else:
        print('Request on URL {} failed with status code {}'.format(link,r.status_code))

soup = get_soup('https://ies.fsv.cuni.cz/content/tree/index/lang/en')

In [44]:
def get_all_news_links(link):
    '''
    Generates list of URLs of all news-related links from the url provided.

    Links on news format: <a href="/en/news/{id}" title="show news" class="show-news">show news</a>

    The URLs are expected in absolute format, i.e. including a full domain.
    '''
    
    soup = get_soup(link)
    
    return ['https://ies.fsv.cuni.cz' + a['href'] for a in soup.find_all('a',{'class':'show-news'})]
links = get_all_news_links('https://ies.fsv.cuni.cz/content/tree/index/lang/en')

In [49]:
soup = get_soup(links[5])

In [50]:
def parse_title(soup):
    '''
    Parse text of the first `h3` object from the soup element.
    '''
    return soup.find('h3').text
parse_title(soup)

'rejustify: a new AI-enhanced database available to our students'

In [54]:
def parse_date(soup):
    '''
    Parse text of the sibling of sibling of the first h3 element in the soup. Note that the immidiate sibling of `h3` is not Tag element, but NavigableString. This is used to represent text between tags.
    '''

    return soup.find('h3').next_sibling.next_sibling.text
pd.Timestamp(parse_date(soup),format='')

Timestamp('2020-02-11 00:00:00')

In [60]:
def parse_news_content(soup):
    '''
    For simplicity, the content of the article is the content of all <p> elements within <div class="col-sm-12 news"></div>

    Return a single string with the whole text. Use `/n` as a connecting string between individual p-texts. 

    Hint: Consider using a `.join()` function applicable on string object
    '''
    div = soup.find('div',{'class':'col-sm-12 news'})
    return '\n \n \n'.join([p.text for p in div.find_all('p')])
print(parse_news_content(soup))

02/11/2020
 
 
Rejustify is an AI-enhanced ETL tool (Extract-Transform-Load) for data set preparation and merging data from multiple sources. The rejustify's algorithm analyzes an empty dataset to recognize its structure and suggests source tables from primary sources of data that match the best. Followed by a "fill" function which sends data queries to the primary sources and fills the required data into the dataset.
 
 
Rejustify has automatized merging and combining data from more than 600 million data series from over 60 most trusted statistical sources like Eurostat, ECB, UN, IMF, statistical offices and central banks, including additional sources like CO2 emissions, Cryptocurrency prices, or COVID-19 cases.
 
 
Rejustify is an online service available through Google Sheets, R and Python. Registration and instruction videos are at rejustify.com and you can contact info@rejustify.com in case of any questions, suggestions and comments.
 
 
Each semester, first 100 students and staff

In [61]:
def parse_ies_news(link,pause=.5):
    '''
    From URL of given news story generate pd.Series object with `title`, `date` and `content`.

    Use functions `parse_title`, `parse_date` and `parse_news_content` to get individual attributes.

    Please, keep the sleep() to prevent overflow of IES website.
    '''

    time.sleep(pause)
    
    soup = get_soup(link)
    return pd.Series({
        'title':parse_title(soup),
        'date':parse_date(soup),
        'content':parse_news_content(soup),
    })
parse_ies_news(links[0])

title       Pre-Defense of Adam Kučera's dissertation thesis
date                                              09/11/2020
content    09/11/2020\n \n \nThe pre-defense of Adam Kuče...
dtype: object

In [62]:
def get_all_news(link):
    '''
    wraping fuctions that accepts a `link` pointing towards hub website with links to parse and returing a dataframe containing all the links
    '''
    links = get_all_news_links(link)
    
    return pd.DataFrame([parse_ies_news(link) for link in links])

In [63]:
news = get_all_news('https://ies.fsv.cuni.cz/content/tree/index/lang/en')
news.head()

,title,date,content
0,Pre-Defense of Adam Kučera's dissertation thesis,09/11/2020,09/11/2020\n \n \nThe pre-defense of Adam Kuče...
1,Job offer at Deloitte,09/11/2020,09/11/2020\n \n \nTeam Valuation & Financial M...
2,European Business Forum online and free of cha...,06/11/2020,"06/11/2020\n \n \nEuropean Business Forum, Dan..."
3,Alumna of the Month in October: Y My Černá Vu,05/11/2020,"05/11/2020\n \n \n\n \n \nIn October, we choos..."
4,Best Global Universities: Charles University h...,03/11/2020,03/11/2020\n \n \nFor the seventh year already...


## Task 4: Convert Task 3 into OOP

In [ ]:
class News:
    def __init__(self,link):
        pass




class Downloader:
    def __init__(self,hub_url):
        pass


dl = Downloader('https://ies.fsv.cuni.cz/content/tree/index/lang/en')
dl.df